In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [32]:
def get_from_binary(year):
    """
    Read in cleaned and trimmed data for SPY_{year} as a numpy array
    Construct a pandas dataframe from the array
    np.fromfile = 0.5 seconds
    pd.read_csv = 30 seconds
    """
    #read in the binary file 
    arr = np.fromfile('Polygon/Primed/SPY_{}.binary'.format(year), dtype=np.float64)
    
    #reshape the 1d array into 2d
    arr = arr.reshape((int(len(arr)/3), 3))

    #construct a Pandas dataframe
    df  = pd.DataFrame(arr)
    
    #reassign the column names
    df.columns = ['t', 'p', 's']
    
    #make the timestamp the index
    df.set_index('t', inplace=True)
   
    #convert 64bit integer index to datetime (unit = nanoseconds)
    df.index = df.index = pd.to_datetime(df.index, unit='ns')
    
    #convert index to US-Eastern timezone, automatically takes care of daylight savings
    df.index = df.index.tz_localize('UTC').tz_convert('US/Eastern')
    
    return df
    
def OHLCV(df, w):
    """
    For the dataframe :df:, calculate the Open,High,Low,Close,Volume every period :w:
    https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
    """
    #resample and calculate OHLCV -- range [w, w+1)
    temp = df.resample(w, label='left',closed='left').agg({'p': 'ohlc', 's': 'sum'})
    temp.columns = ['open','high','low','close','volume']
    
    #forward fill the close price to fill in any missing values
    temp['close'].fillna(method='ffill', inplace=True)
    temp['open'].fillna(temp['close'], inplace=True)
    temp['high'].fillna(temp['close'], inplace=True)
    temp['low'].fillna(temp['close'], inplace=True)
    
    return temp

def bivariate(df, w):
    """
    Calculate the last price and sum of shares traded in each period w
    """
    #resample and calculate last price and sum of shares (volume) for each period :w:
    #range [w, w+1)
    temp = df.resample(w, label='left',closed='left').agg({'p':'last','s':'sum'})
    
    #forward fill the price for missing values
    temp['p'].fillna(method='ffill', inplace=True)
    
    return temp

def aggregate_trades(df, agg_type, w):

    #get aggregated data for the eval_period
    if agg_type == 'ohlcv':
        agg_data = OHLCV(df, w)
                
    elif agg_type == 'bivariate':
        agg_data = bivariate(df, w)
    
    #fill in zero shares with one share so that transform(df) doesn't divide by zero
    agg_data.loc[agg_data['s'] < 1, 's'] = 1.0
    
    return agg_data
    

def transform(df):
    """
    Convert prices and volume to percent different from previous
    """
    #calculate percent difference from previous :w:
    df['p'] = df['p'].pct_change()
    df['s'] = df['s'].pct_change()
    
    #take natural log of returns to get normal distribution
    df['s'] = np.log(1 + df['s'])
    df['p'] = np.log(1 + df['p'])
    
    return df

def strided_window(arr, window_size, step_size):
    """
    Create expanded array of references that effectively create a "memory" for each row,
    where columns are previous rows' data.
    See generate_view() for information on what window_size and step_size are
    https://stackoverflow.com/questions/40084931/taking-subarrays-from-numpy-array-with-given-stride-stepsize/40085052#40085052
    """
    #needed to properly calculate stride sizes, can corrupt memory if incorrect
    nrows = ((arr.size-window_size) // step_size) + 1
    n     = arr.strides[0]
    
    #use striding tricks to create training data -- prevent writing to array to reduce chance of corruption
    data_references = np.lib.stride_tricks.as_strided(arr, shape=(nrows,window_size), strides=(step_size*n,n), writeable=False)
    
    return data_references

def generate_view(aggregated_data, lookback):
    """
    Generates numpy view of ohlcv_data with shape (nrows-lookback, ncols*lookback)
    :aggregated_data: a contiguous numpy array with shape (n,m)
    :lookback: an integer specifying how many previous periods (w) to include in each row
    """
    #increment lookback to produce expected behavior
    lookback = lookback + 1
    
    ncols = aggregated_data.shape[1]
    
    #make the data a 1-dimensional array (unravel it)
    data  = aggregated_data.ravel()
    
    #get views (references) of data (no copying, no extra memory)
    data_strided = strided_window(data, window_size = lookback*ncols, step_size = ncols)
    
    return data_strided


def parse_args(agg_type, w, lookback, eval_period):
    """
    string args passed by tf.data.Dataset.from_generator are binary, must be decoded
    also performs checks to make sure args are valid
    """
    try:
        agg_type    = agg_type.decode('utf-8').lower().strip()
        w           = w.decode('utf-8')
        eval_period = eval_period.decode('utf-8')
    except:
        pass
    
    assert agg_type in ['ohlcv', 'bivariate']
    assert lookback >= 0

    return agg_type, w, lookback, eval_period

def data_generator(agg_type, w, lookback, eval_period, numpy_vals = True):
    """
    Yield either OHLCV or bivariate data aggregated at w
    Each yield is data for one eval_period
    Reward (profit) should be assessed at end of each eval_period
    microseconds(U) | milliseconds(L) | seconds(S) | minutes(T) | hour(H)
    """
    agg_type, w, lookback, eval_period = parse_args(agg_type, w, lookback, eval_period)
    
    #go one year at a time to reduce memory usage -- reading in CSV's is a bottleneck
    for year in range(2003,2005):
        print(year)
        
        #read in the cleaned and trimmed trade data
        spy = get_from_binary(year)
        
        #start iteration count
        first_iteration = True
        
        #for each evaluation period (i.e. each game of pong) get aggregated data
        for name, eval_period_data in spy.groupby(pd.Grouper(freq=eval_period)):

            #if weekend or other time period with no trades
            if eval_period_data.shape[0] <= lookback:
                continue
                
            print('\t{}'.format(name))                

            ## get X ##
            #get aggregated data
            agg_data = aggregate_trades(eval_period_data, agg_type, w)

            ## get Y ##
            Prices = np.array(agg_data['p'].values[1:].astype(np.float32))

            ## get Z ##
            Mark = agg_data['p'].values[0]
            
            #transform the data (log returns)
            agg_data = transform(agg_data)
            agg_data = agg_data.iloc[1:,]

            #yield Xtrain, Ytrain of previous window
            if not first_iteration:
                yield Xtrain, Ytrain, Mark
            
            #update to this window
            Xtrain = generate_view(agg_data.values, lookback).astype(np.float32)
            Ytrain = Prices.reshape((Prices.shape[0],1))
            
            first_iteration = False

In [ ]:
mygen1 = data_generator(agg_type    = 'bivariate', 
                        w           = 'S',
                        lookback    = 0,
                        eval_period = 'D',
                        numpy_vals  = False)
tmp = next(mygen1)[0]
tmp

In [34]:
mygen2 = data_generator(agg_type    = 'bivariate', 
                        w           = '10S',
                        lookback    = 0,
                        eval_period = 'min',
                        numpy_vals  = True)
Xt, Yt, Zt = next(mygen2)
print(Xt)
print(Yt)
print(Zt)

2003
	2003-12-01 09:30:00-05:00
	2003-12-01 09:31:00-05:00
[[ 0.0000000e+00 -4.4067192e+00]
 [ 7.4845011e-04  5.8774557e+00]
 [-2.8058531e-04 -1.9815620e+00]
 [ 0.0000000e+00 -1.3781972e+00]
 [ 2.8058531e-04  1.0376515e+00]]
[[106.85]
 [106.93]
 [106.9 ]
 [106.9 ]
 [106.93]]
106.93000030517578


In [ ]:
##TODO: how to get first price in next time period window? save previous data and execute with lag

In [ ]:
fig = plt.figure()
plt.plot(tmp.p)
fig.suptitle('Natural log of percent difference in prices over time', fontsize=16)
plt.axis('off')
print()

In [ ]:
fig = plt.figure()
plt.plot(tmp.s)
fig.suptitle('Natural log of percent difference in volume over time', fontsize=16)
plt.axis('off')
print()

In [ ]:
fig = plt.figure()
plt.hist(tmp.p, bins = 101)
fig.suptitle('Histogram of natural log of percent difference in prices', fontsize=16)
plt.yscale('log')
plt.xlabel('ln ( pct difference )', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
print()

In [ ]:
fig = plt.figure()
plt.hist(tmp.s, bins = 109)
fig.suptitle('Histogram of natural log of percent difference in volume', fontsize=16)
plt.yscale('log')
plt.xlabel('ln ( pct difference )', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
print()

In [ ]:
TYPE      = 'bivariate'
W         = 'T' #minute
LOOKBACK  = 2
EVAL      = 'D' #day


bivariate_generator = tf.data.Dataset.from_generator(
                    generator     = data_generator, args=[TYPE,W,LOOKBACK,EVAL], 
                    output_types  = (tf.float32, tf.float32),
                    output_shapes =(tf.TensorShape([2*(LOOKBACK+1) if TYPE == 'bivariate' else 5*(LOOKBACK+1)]), tf.TensorShape([]))
                    ).repeat().batch(8).prefetch(8)
     
price_generator

Need an input specifying current position

# Try the Tensorflow custom training example

## My trade data

## The neural network

In [35]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(2,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(1, activation=tf.nn.tanh)
])
print('Test run output sentiments:')
model(Xt)

Test run output sentiments:


<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[0.9318945 ],
       [0.63839996],
       [0.6362751 ],
       [0.48009208],
       [0.1325867 ]], dtype=float32)>

## The loss

### Custom Loss 1 (agnostic of current position)

In [37]:
def loss(model, market, prices, mark, training, debug=True):
    sentiments = model(market, training=training)
    position   = tf.reduce_sum(sentiments)
    avg_price  = tf.reduce_sum(tf.multiply(sentiments, prices)) / position
    diff       = tf.subtract(mark, avg_price)
    profit     = tf.multiply(diff, position)
    loss       = -profit
    if debug:
        print('\tAvg Price: {}'.format(avg_price))
        print('\tPosition: {}'.format(position))
        print('\tProfit: {}'.format(profit))
    return loss
print('Loss: {}'.format(loss(model, Xt, Yt, Zt, training=False, debug=True)))


	Avg Price: 106.89168548583984
	Position: 2.8192481994628906
	Profit: 0.10801898688077927
Loss: -0.10801898688077927


## The gradients

In [41]:
def grad(model, market, prices, mark):
    with tf.GradientTape() as tape:
        loss_value = loss(model, market, prices, mark, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)


def forward_pass_positions(model, inputs):
    position   = 0
    sentiments = np.zeros((inputs.shape[0]))
    
    for i in range(0,inputs.shape[0]-1):
        #get sentiment for single observation (position = 0)
        s         = model(inputs[i-1])[0][0] 
        position += s
        
        #fill in position for next observation
        inputs[i+1][0][2] = position
        sentiments[i]      = s
        
    #get sentiment for last observation
    sentiments[-1] = model(inputs[-1])[0][0] 
    return sentiments

def custom_grad(model, inputs, prices):
    #do non-differentiable work here:
    sentiments = forward_pass_positions(model, inputs)
    
    #differentiable work (calculate gradients)
    with tf.GradientTape() as tape:
        loss_value = custom_loss_2(model, sentiments, prices, training=True)
        
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

## The optimizer

In [42]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

#Example of training step
loss_value, grads = grad(model, Xt, Yt, Zt) #last arg is labels

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))


print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),loss(model, Xt, Yt, Zt, training=True).numpy()))


	Avg Price: 106.89168548583984
	Position: 2.8192481994628906
	Profit: 0.10801898688077927
Step: 0, Initial Loss: -0.10801898688077927
	Avg Price: 106.89189147949219
	Position: 2.920367479324341
	Profit: 0.11129177361726761
Step: 1,         Loss: -0.11129177361726761


## Training

In [43]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 50

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()

    # Training loop - using batches of batch_size
    #for x, y in train_dataset:
        
        #print('x: {}'.format(x))
        #print('y: {}'.format(y))
        
    # Optimize the model
    loss_value, grads = grad(model, Xt, Yt, Zt)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg(loss_value)  # Add current batch loss

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())

    #if epoch % 50 == 0:
    print('Epoch {:03d}: Loss: {:.3f}'.format(epoch+1, epoch_loss_avg.result()))

	Avg Price: 106.89189147949219
	Position: 2.920367479324341
	Profit: 0.11129177361726761
Epoch 001: Loss: -0.111
	Avg Price: 106.8921127319336
	Position: 3.0059540271759033
	Profit: 0.11388830095529556
Epoch 002: Loss: -0.114
	Avg Price: 106.89228820800781
	Position: 3.0796096324920654
	Profit: 0.11613854020833969
Epoch 003: Loss: -0.116
	Avg Price: 106.8924789428711
	Position: 3.143947124481201
	Profit: 0.11796517670154572
Epoch 004: Loss: -0.118
	Avg Price: 106.89262390136719
	Position: 3.200815439224243
	Profit: 0.1196349710226059
Epoch 005: Loss: -0.120
	Avg Price: 106.89279174804688
	Position: 3.251528263092041
	Profit: 0.12098467350006104
Epoch 006: Loss: -0.121
	Avg Price: 106.8929443359375
	Position: 3.2971112728118896
	Profit: 0.12217765301465988
Epoch 007: Loss: -0.122
	Avg Price: 106.89308166503906
	Position: 3.3383612632751465
	Profit: 0.12324775755405426
Epoch 008: Loss: -0.123
	Avg Price: 106.8931884765625
	Position: 3.3759098052978516
	Profit: 0.12427341192960739
Epoch 0